In [1]:
import numpy as np
import tensorflow as tf
from bokeh.palettes import Category10
from bokeh.plotting import figure, output_notebook, show
from utilities import *
from shallow_selector import *
np.random.seed(1000)

# Output to Jupyter notebook
output_notebook()
np.seterr(all='warn')

2024-04-22 13:31:35.097277: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 13:31:35.652571: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading BokehJS ...

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
def plot_data_space(data, labels, title="Input Data, color-coded by label"):
    x = data[:, 0]
    y = data[:, 1]

    # Extract the unique values and the reconstruction map
    u, inv = np.unique(labels, return_inverse=True)
    
    # Create a new plot with a title and axis labels
    p = figure(title=title, x_axis_label='Dimension 1', y_axis_label='Dimension 2')
    
    # Add a circle glyph
    num_colors = max(len(u), 3)
    colors = [Category10[num_colors][e] for e in inv]
    p.circle(x, y, size=10, color=colors, alpha=0.5)
    
    # Show the plot
    show(p)

In [3]:
def plot_data_evolution(base, delta_over_time, labels, title="Input Data, color-coded by label"):
    # Extract the unique values and the reconstruction map
    u, inv = np.unique(labels, return_inverse=True)
    gapT = 50
    
    # Create a new plot with a title and axis labels
    p = figure(title=title, x_axis_label='Dimension 1', y_axis_label='Dimension 2')
    
    # Add a circle glyph
    num_colors = max(len(u), 3)
    colors = [Category10[num_colors][e] for e in inv]

    # Plot base points
    x = base[:, 0]
    y = base[:, 1]
    p.circle(x, y, size=10, color=colors, alpha=0.5)

    # Plot evolution
    num_keys = max(delta_over_time.keys())
    for i in range(len(y)):
        trajx = [delta_over_time[j][i, 0] for j in range(0, num_keys, gapT)]
        trajy = [delta_over_time[j][i, 1] for j in range(0, num_keys, gapT)]
        p.line(trajx, trajy, line_color=colors[i], alpha=0.2)
    
    # Show the plot
    show(p)

In [4]:
X_train = generate_uniform_unit_hypersphere(1000, 2)
y_train = np.array([1 if pt[1] >= -pt[0] else 0 for pt in X_train]).astype(np.float64)
plot_data_space(X_train, y_train, title="Training data, color-coded by label")

In [5]:
X_eval = generate_grid_unit_hypersphere(0.077, 2)
y_eval = np.array([1 if pt[0] > 0 else 0 for pt in X_eval]).astype(np.float64)
plot_data_space(X_eval, y_eval, title="Evaluation data, color-coded by label")

In [6]:
evaluate(X_train, y_train, X_eval, y_eval)

2024-04-22 13:31:37.254121: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-22 13:31:37.254565: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Precision: 0.7503462544671101, Accuracy: 0.75
Times correct: 0.75


(0.7503462544671101, 0.75)

## Premise
Within the context of data classification through stochastic gradient descent (SGD), Ma et al argue that we can inutively find the best $k$ items to poison in a dataset by selecting them from data elements with the largest initial gradient norms

$$
\Big\| \frac{\partial J(\tilde{D})}{\partial\tilde{z}_i} \Big|_{\tilde{D}=D}\Big\|
$$

where $D$ is your dataset, $\tilde{D}$ is your poisoned dataset, and $J$ is your attack objective function (detailed in the next section).

## Implementation Approach

Given:
- Input features: $X = \{x_1, x_2, \ldots, x_n\}$
- Labels: $Y = \{y_1, y_2, \ldots, y_n\}$
- Regularization parameter: $\lambda$

**Step 1: Compute the weights for the logistic regression model:**

1. Define the logistic regression objective function $J(w)$ as:
   $$
   J(w) = \frac{1}{n}\sum_{i=1}^{n} \log(1 + \exp(-y_i \cdot (x_i^T \cdot w))) + \frac{\lambda}{2} ||w||^2
   $$
   where:
   - $w$ is the vector of weights
   - $x_i$ is the $i$-th feature vector
   - $y_i$ is the $i$-th label
   as was performed in Section 5.3's Label Aversion attack.

2. Initialize weights $w$ to zero.

3. Optimize $J(w)$ using stochastic gradient descent:
   $$
   \text{Repeat until convergence:} \quad w = w - \alpha \cdot \nabla J(w)
   $$
   where $\alpha$ is the learning rate.

4. Return the computed weights $w$.

**Step 2: Compute the gradient approximation for item selection:**

1. Compute the exponentials for each sample:
   $$
   \text{exp}_i = \exp(-y_i \cdot (x_i^T \cdot w))
   $$

2. Compute the diagonal matrix $S$:
   $$
   S = \text{Diag}\left(\frac{\text{exp}_i}{(1 + \text{exp}_i)^2}\right)
   $$

3. Compute the inverse of the regularization matrix:
   $$
   R_{\text{inv}} = (\lambda \cdot I + X^T \cdot S \cdot X)^{-1}
   $$

4. Compute the gradient approximation for each sample:
   $$
   \nabla_{\text{approx}_i} = R_{\text{inv}} \cdot \frac{\partial w}{\partial x_i}
   $$

5. Return the stack of gradient approximations.

**Step 3: Perform shallow selection of top-k items:**

1. Compute the norm of each gradient approximation: $||\nabla_{\text{approx}_i}||$.

2. Sort the norms in descending order.

3. Select the top-$k$ indices corresponding to the largest norms.

4. Return the selected indices.


In [7]:
regularization = 10  # From Ma et al's paper
k = 200

## Sanity Check: K-item selection via literature approach vs randomized selection

In [8]:
# Instantiate the model
model = ShallowSelector(regularization)
target_idx = model.shallow_selection(X_train, y_train, k)
target_idx = target_idx.numpy()

bad_x, bad_y = apply_negation(X_train, y_train, target_idx)

plot_data_space(bad_x, bad_y, title="Shallow Selection on SV learner with Label Aversion Heuristic")
evaluate(bad_x, bad_y, X_eval, y_eval)

Target value for label 0.0 is [0.86562939 0.49459467]
Target value for label 1.0 is [-0.62251507 -0.43306061]


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Precision: 0.7501067664292181, Accuracy: 0.75
Times correct: 0.75


(0.7501067664292181, 0.75)

In [9]:
target_idx = np.random.choice(np.arange(X_train.shape[0]), k)
bad_x, bad_y = apply_negation(X_train, y_train, target_idx)
plot_data_space(bad_x, bad_y, title="Random Selection with Label Aversion Heuristic")
evaluate(bad_x, bad_y, X_eval, y_eval)

Target value for label 0.0 is [0.86562939 0.49459467]
Target value for label 1.0 is [-0.78030552 -0.59825287]


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Precision: 0.7439516129032258, Accuracy: 0.743801652892562
Times correct: 0.743801652892562


(0.7439516129032258, 0.743801652892562)

## Sanity Check: Literature attack using literature K-item selection

In [10]:
model = ShallowSelector(regularization)
target_idx = model.shallow_selection(X_train, y_train, k)
evolution, poisoned = model.attack(X_train, y_train, target_idx, X_eval, y_eval, 500, 0.0001, 1.0)

Start running label-aversion attack...
100
200
300
400


In [11]:
plot_data_evolution(poisoned, evolution, y_train, title="Shallow Selection with Label Aversion on Surrogate Victims")

In [12]:
evaluate(poisoned, y_train, X_eval, y_eval)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Precision: 0.7463768115942029, Accuracy: 0.7458677685950413
Times correct: 0.7458677685950413


(0.7463768115942029, 0.7458677685950413)

## Comparison of All Techniques on Generated Data

In [13]:
# This takes a very long time (~3hrs)
regularization = 10
epochs = 1000
scores = {"shallow-sv" : [], "random-sv" : [], "shallow-heuristic" : [], "random-heuristic" : [], "k": list(range(50, 1000, 50))}

# Get baseline score
ppv, acc = evaluate(X_train, y_train, X_eval, y_eval)
scores["baseline"] = [acc] * len(scores["k"])

for k in scores["k"]:
    print(f"EVALUATING FOR K={k}")
    model = ShallowSelector(regularization)

    # SHALLOW-Label-Aversion-on-SVs
    target_idx = model.shallow_selection(X_train, y_train, k)
    evolution, poisoned_x = model.attack(X_train, y_train, target_idx, X_eval, y_eval, epochs, 0.0001, 1.0)
    ppv, acc = evaluate(poisoned_x, y_train, X_eval, y_eval)
    scores["shallow-sv"].append(acc)

    # RANDOM-Label-Aversion-on-SVs
    target_idx = np.random.choice(np.arange(X_train.shape[0]), k)
    evolution, poisoned_x = model.attack(X_train, y_train, target_idx, X_eval, y_eval, epochs, 0.0001, 1.0)
    ppv, acc = evaluate(poisoned_x, y_train, X_eval, y_eval)
    scores["random-sv"].append(acc)

    # SHALLOW-label-heuristic
    target_idx = model.shallow_selection(X_train, y_train, k)
    target_idx = target_idx.numpy()
    bad_x, bad_y = apply_negation(X_train, y_train, target_idx)
    ppv, acc = evaluate(bad_x, bad_y, X_eval, y_eval)
    scores["shallow-heuristic"].append(acc)

    # RANDOM-label-heuristic
    target_idx = np.random.choice(np.arange(X_train.shape[0]), k)
    bad_x, bad_y = apply_negation(X_train, y_train, target_idx)
    evaluate(bad_x, bad_y, X_eval, y_eval)
    ppv, acc = evaluate(bad_x, bad_y, X_eval, y_eval)
    scores["random-heuristic"].append(acc)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Precision: 0.7503462544671101, Accuracy: 0.75
Times correct: 0.75
EVALUATING FOR K=50
Start running label-aversion attack...
100
200
300
400
500
600
700
800
900
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Precision: 0.7503462544671101, Accuracy: 0.75
Times correct: 0.75
Start running label-aversion attack...
100
200
300
400
500
600
700
800
900
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Precision: 0.735940065681445, Accuracy: 0.7355371900826446
Times correct: 0.7355371900826446
Target value for label 0.0 is [0.86562939 0.49459467]
Target value for label 1.0 is [-0.76518094 -0.53910116]
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Precision: 0.7482051282051282, Accuracy: 0.7479338842975206
Times correct: 0.7479338842975206
Target value for label 0.0 is [0.86562939 0.49459467]
Target value for label 1.0 is [-0.78030552 -0.59825287]
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Precision: 0.746208299847819, Accuracy: 0.7458677685950413
Times correct: 0.745867768595041

In [14]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, NumeralTickFormatter

source = ColumnDataSource(data=scores)

# Create a figure
p = figure(title="Model Performance on Uniform Data", x_axis_label='k', y_axis_label='Accuracy', width=600, height=300)

# Plot lines
p.line(x='k', y='shallow-sv', legend_label='Shallow (SV)', line_width=2, color=Category10[5][0], source=source)
p.line(x='k', y='random-sv', legend_label='Random (SV)', line_width=2, color=Category10[5][1], source=source)
p.line(x='k', y='shallow-heuristic', legend_label='Shallow (Heuristic)', line_width=2, color=Category10[5][2], source=source)
p.line(x='k', y='random-heuristic', legend_label='Random (Heuristic)', line_width=2, color=Category10[5][3], source=source)
p.line(x='k', y='baseline', legend_label='Baseline', line_width=2, color=Category10[5][4], source=source)
p.legend.location = "bottom_left"
p.legend.click_policy="hide"
p.yaxis.formatter = NumeralTickFormatter(format='0.0%')

# Show the plot
show(p)

## Comparison of All Techniques on Synthea Covid-19 Data

In [15]:
import os
data_dir = ''
with open(os.path.join(data_dir, 'X.txt'), 'r') as f:
    X_train = np.loadtxt(f)
with open(os.path.join(data_dir, 'y.txt'), 'r') as f:
    y_train = np.loadtxt(f)
with open(os.path.join(data_dir, 'Xeval.txt'), 'r') as f:
    X_eval = np.loadtxt(f)
with open(os.path.join(data_dir, 'yeval.txt'), 'r') as f:
    y_eval = np.loadtxt(f)

In [ ]:
# This takes a VERY long time (~5hrs)
regularization = 10
epochs = 500
scores = {"shallow-sv" : [], "random-sv" : [], "shallow-heuristic" : [], "random-heuristic" : [], "k": list(range(100, 1000, 100))}

# Get baseline score
ppv, acc = evaluate(X_train, y_train, X_eval, y_eval)
scores["baseline"] = [acc] * len(scores["k"])

for k in scores["k"]:
    print(f"EVALUATING FOR K={k}")
    model = ShallowSelector(regularization)

    # SHALLOW-Label-Aversion-on-SVs
    target_idx = model.shallow_selection(X_train, y_train, k)
    evolution, poisoned_x = model.attack(X_train, y_train, target_idx, X_eval, y_eval, epochs, 0.0001, 1.0)
    ppv, acc = evaluate(poisoned_x, y_train, X_eval, y_eval)
    scores["shallow-sv"].append(acc)

    # RANDOM-Label-Aversion-on-SVs
    target_idx = np.random.choice(np.arange(X_train.shape[0]), k)
    evolution, poisoned_x = model.attack(X_train, y_train, target_idx, X_eval, y_eval, epochs, 0.0001, 1.0)
    ppv, acc = evaluate(poisoned_x, y_train, X_eval, y_eval)
    scores["random-sv"].append(acc)

    # SHALLOW-label-heuristic
    target_idx = model.shallow_selection(X_train, y_train, k)
    target_idx = target_idx.numpy()
    bad_x, bad_y = apply_negation(X_train, y_train, target_idx)
    ppv, acc = evaluate(bad_x, bad_y, X_eval, y_eval)
    scores["shallow-heuristic"].append(acc)

    # RANDOM-label-heuristic
    target_idx = np.random.choice(np.arange(X_train.shape[0]), k)
    bad_x, bad_y = apply_negation(X_train, y_train, target_idx)
    evaluate(bad_x, bad_y, X_eval, y_eval)
    ppv, acc = evaluate(bad_x, bad_y, X_eval, y_eval)
    scores["random-heuristic"].append(acc)

104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step
Precision: nan, Accuracy: 0.875151515151515
Times correct: 0.7503030303030302
EVALUATING FOR K=100


/home/ben/repositories/bias-bombs/utilities.py:96: RuntimeWarning: invalid value encountered in divide
  PPV = TP/(TP+FP)
/home/ben/repositories/bias-bombs/utilities.py:98: RuntimeWarning: invalid value encountered in divide
  NPV = TN/(TN+FN)
/home/ben/repositories/bias-bombs/utilities.py:104: RuntimeWarning: invalid value encountered in divide
  FDR = FP/(TP+FP)


Start running label-aversion attack...
100
200
300
400
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 504us/step
Precision: nan, Accuracy: 0.875151515151515
Times correct: 0.7503030303030302
Start running label-aversion attack...


/home/ben/repositories/bias-bombs/utilities.py:96: RuntimeWarning: invalid value encountered in divide
  PPV = TP/(TP+FP)
/home/ben/repositories/bias-bombs/utilities.py:98: RuntimeWarning: invalid value encountered in divide
  NPV = TN/(TN+FN)
/home/ben/repositories/bias-bombs/utilities.py:104: RuntimeWarning: invalid value encountered in divide
  FDR = FP/(TP+FP)


100
200
300
400
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 501us/step
Precision: nan, Accuracy: 0.875151515151515
Times correct: 0.7503030303030302


...

/home/ben/repositories/bias-bombs/utilities.py:96: RuntimeWarning: invalid value encountered in divide
  PPV = TP/(TP+FP)
/home/ben/repositories/bias-bombs/utilities.py:98: RuntimeWarning: invalid value encountered in divide
  NPV = TN/(TN+FN)
/home/ben/repositories/bias-bombs/utilities.py:104: RuntimeWarning: invalid value encountered in divide
  FDR = FP/(TP+FP)


104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 515us/step
Precision: nan, Accuracy: 0.875151515151515
Times correct: 0.7503030303030302
EVALUATING FOR K=200


/home/ben/repositories/bias-bombs/utilities.py:96: RuntimeWarning: invalid value encountered in divide
  PPV = TP/(TP+FP)
/home/ben/repositories/bias-bombs/utilities.py:98: RuntimeWarning: invalid value encountered in divide
  NPV = TN/(TN+FN)
/home/ben/repositories/bias-bombs/utilities.py:104: RuntimeWarning: invalid value encountered in divide
  FDR = FP/(TP+FP)


Start running label-aversion attack...
100
200
300
400
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 508us/step
Precision: nan, Accuracy: 0.875151515151515
Times correct: 0.7503030303030302
Start running label-aversion attack...


/home/ben/repositories/bias-bombs/utilities.py:96: RuntimeWarning: invalid value encountered in divide
  PPV = TP/(TP+FP)
/home/ben/repositories/bias-bombs/utilities.py:98: RuntimeWarning: invalid value encountered in divide
  NPV = TN/(TN+FN)
/home/ben/repositories/bias-bombs/utilities.py:104: RuntimeWarning: invalid value encountered in divide
  FDR = FP/(TP+FP)


In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, NumeralTickFormatter

source = ColumnDataSource(data=scores)

# Create a figure
p = figure(title="Model Performance", x_axis_label='k', y_axis_label='Accuracy', width=600, height=300)

# Plot lines
p.line(x='k', y='shallow-sv', legend_label='Shallow', line_width=2, color=Category10[5][0], source=source)
p.line(x='k', y='random-sv', legend_label='Random (SV)', line_width=2, color=Category10[5][1], source=source)
p.line(x='k', y='shallow-heuristic', legend_label='Random (Heuristic)', line_width=2, color=Category10[5][2], source=source)
p.line(x='k', y='random-heuristic', legend_label='Baseline (Heuristic)', line_width=2, color=Category10[5][3], source=source)
p.line(x='k', y='baseline', legend_label='Baseline (SV)', line_width=2, color=Category10[5][4], source=source)
p.legend.location = "bottom_left"
p.legend.click_policy="hide"
p.yaxis.formatter = NumeralTickFormatter(format='0.0%')
# Strong relationship
# Show the plot
show(p)

In [ ]:
model = ShallowSelector(100)
target_idx = model.shallow_selection(X_train, y_train, 5000)
evolution, poisoned_x = model.attack(X_train, y_train, target_idx, X_eval, y_eval, 300, 0.0001, 0.5)
ppv, acc = evaluate(poisoned_x, y_train, X_eval, y_eval)
plot_data_evolution(poisoned_x, evolution, y_train)